In [2]:
# 1) Set the location
%cd ../../code/

/Users/user/Documents/GitHub/paraphrase_py/code


/Users/user/Documents/GitHub/Video-from-Script/youtube_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [15]:
import os
import pandas as pd

from pathlib import Path
from read_and_write_docs import read_jsonl, write_jsonl

In [19]:
root_dir = Path("/Volumes/BCross/temp_datasets/Wiki_sample/paraphrased_clean/")
out_path = "/Volumes/BCross/temp_datasets/Wiki_sample/cleaned_paraphrases.jsonl"

In [17]:
frames   = []

for current_dir, _, files in os.walk(root_dir):
    current_path = Path(current_dir)
    rel_subdir   = current_path.relative_to(root_dir)          # "" for top level

    for file_name in files:
        if not file_name.endswith(".jsonl"):
            continue

        file_path = current_path / file_name
        df        = read_jsonl(file_path)                      # ← your helper

        # 1️⃣  put the sub-directory string into generation_type
        gen_type  = "" if rel_subdir == Path(".") else str(rel_subdir)
        df.insert(0, "generation_type", gen_type)

        # 2️⃣  derive gen_category once and broadcast to the whole DataFrame
        if "qwen_gen" in gen_type:
            category = "generation"
        elif "qwen_para" in gen_type:
            category = "paraphrase"
        else:
            category = "paraphrase less strict"

        df.insert(1, "gen_category", category)                 # keep it next to generation_type
        frames.append(df)

# 3️⃣  stitch everything together
big_df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

In [18]:
big_df

,generation_type,gen_category,doc_id,orig_doc_id,corpus,author,texttype,text,generated_text,top_p,temperature,time_sec,tokens_per_sec,clean_text,text_cleaned,clean_stage,parsing_errors
0,qwen_gen_t0.7_tp0.9,generation,a_man_in_black_text_2,known [A_Man_In_Black - Text-2].txt,Wiki,A_Man_In_Black,known,"Meaning not DMM/Arzon/whatever, or Amazon.co.j...","```json\n{\n ""new_document"": ""In recent month...",0.9,0.7,11.094461,121.051395,"In recent months, discussions surrounding the ...",1,fix_salvage_quotes,[original: Expecting value: line 1 column 1 (c...
1,qwen_gen_t0.7_tp0.9,generation,a_man_in_black_text_2,known [A_Man_In_Black - Text-2].txt,Wiki,A_Man_In_Black,known,"Meaning not DMM/Arzon/whatever, or Amazon.co.j...","```json\n{\n ""new_document"": ""Image licensing...",0.9,0.7,10.675884,125.797544,Image licensing is a contentious topic among m...,1,fix_pythonic,[original: Expecting value: line 1 column 1 (c...
2,qwen_gen_t0.7_tp0.9,generation,a_man_in_black_text_2,known [A_Man_In_Black - Text-2].txt,Wiki,A_Man_In_Black,known,"Meaning not DMM/Arzon/whatever, or Amazon.co.j...","```json\n{""new_document"": ""A dedicated fan gam...",0.9,0.7,10.283379,129.334915,A dedicated fan game enthusiast often faces ch...,1,fix_salvage_quotes,[original: Expecting value: line 1 column 1 (c...
3,qwen_gen_t0.7_tp0.9,generation,a_man_in_black_text_2,known [A_Man_In_Black - Text-2].txt,Wiki,A_Man_In_Black,known,"Meaning not DMM/Arzon/whatever, or Amazon.co.j...","{""new_document"": ""As mentioned earlier, the pr...",0.9,0.7,4.166524,267.369146,"As mentioned earlier, the primary focus of thi...",1,fix_salvage_quotes,[original: Extra data: line 1 column 770 (char...
4,qwen_gen_t0.7_tp0.9,generation,a_man_in_black_text_2,known [A_Man_In_Black - Text-2].txt,Wiki,A_Man_In_Black,known,"Meaning not DMM/Arzon/whatever, or Amazon.co.j...","```json\n{\n ""new_document"": ""It appears that...",0.9,0.7,7.619262,162.220433,It appears that your article discusses the cha...,1,fix_salvage_quotes,[original: Expecting value: line 1 column 1 (c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,qwen_para_t1.5_tp0.9,paraphrase,headleydown_text_3,known [HeadleyDown - Text-3].txt,Wiki,HeadleyDown,known,You have just made undiscussed and unagreed ch...,"```json\n{\n ""new_document"": ""In light of pre...",0.9,1.5,8.147563,171.216851,"{\n ""new_document"": ""In light of previously a...",1,wrap_plain_text,[original: Expecting value: line 1 column 1 (c...
1096,qwen_para_t1.5_tp0.9,paraphrase,headleydown_text_3,known [HeadleyDown - Text-3].txt,Wiki,HeadleyDown,known,You have just made undiscussed and unagreed ch...,"```json\n{\n ""new_document"": [\n ""Just not...",0.9,1.5,12.139852,126.195933,Just noticed that the article underwent unfore...,1,fix_salvage_quotes,[original: Expecting value: line 1 column 1 (c...
1097,qwen_para_t1.5_tp0.9,paraphrase,headleydown_text_3,known [HeadleyDown - Text-3].txt,Wiki,HeadleyDown,known,You have just made undiscussed and unagreed ch...,"{\n ""new_document"": ""Inundated with unauthori...",0.9,1.5,8.723459,162.206305,Inundated with unauthorized and unreconciled a...,1,fix_salvage_quotes,[original: Expecting property name enclosed in...
1098,qwen_para_t1.5_tp0.9,paraphrase,headleydown_text_3,known [HeadleyDown - Text-3].txt,Wiki,HeadleyDown,known,You have just made undiscussed and unagreed ch...,"{\n ""new_document"": ""You have recently altere...",0.9,1.5,9.667208,149.681278,You have recently altered parts of our draft.\...,1,fix_salvage_quotes,"[original: Expecting ',' delimiter: line 4 col..."


In [20]:
write_jsonl(big_df, out_path)

In [21]:
# This is for after doing the parascore
# df = read_jsonl('/Volumes/BCross/temp_datasets/Wiki_sample/parascore.jsonl')
# df.to_excel("/Volumes/BCross/temp_datasets/Wiki_sample/wiki_examples.xlsx", index=False)